# Extraction of Collocated ESA CCI Biomass and Sentinel-2 NDVI

In [1]:
import os

# define repo name and get root working directory
repo = 'byoc-api'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
root_path

'C:\\Users\\crwil\\Documents\\GitHub\\sentinelhub\\byoc-api'

In [2]:
# get pathname to configuration file
cfg_file = os.path.join( os.path.join( root_path, 'cfg' ), 'fusion/s2-agb.yml' )
with open( cfg_file ) as f:
    print ( f.read() )

request:
    evalscript: |
                //VERSION=3
                function setup() {
                    return {
                        input: [{
                            datasource: "byoc",
                            bands: [ "agb", "agb_sd" ]
                        },
                        {
                            datasource: "s2_l2a",
                            bands: [ "B04", "B08", "SCL" ],
                        } ],
                        output: [
                            {id: "AGB", bands: 1, sampleType: SampleType.UINT16},
                            {id: "AGB_SD", bands: 1, sampleType: SampleType.UINT16},
                            {id: "NDVI", bands: 1, sampleType: SampleType.FLOAT32} 
                        ]
                    }
                }
                function evaluatePixel(samples) {                                  
                    
                    var byoc = samples.byoc[0]
                    var s2 = samples.s2_l2a[0]
  

In [3]:
import yaml
from munch import munchify

# load cfg file using yaml parser
with open( cfg_file, 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )

### _Arbitrary location of a land parcel somewhere in southwest Kenya_

In [4]:
# setup aoi limits and resolution
coords = 35, -0.9, 36, 0.0
resolution = 100

In [5]:
# create instance of shclient class
from processapi import Client
client = Client( config )

In [6]:
# get utm bbox
bbox = client.getBoundingBox( coords )
print ( bbox, bbox.crs.value )

722568.1399615373,9900461.841652302,833978.5569194595,10000000.0 32736


In [7]:
# get aoi image dimensions
client.getBoxDimensions( bbox, resolution )

(1114, 995)

### _Define and extract biomass + S2 time-series for autumn 2018_

In [8]:
# pick a few dates
from datetime import datetime
timeframe = { 'start' : datetime( 2018, 9, 1, 0, 0, 0 ), 
                'end' : datetime( 2018, 11, 7, 23, 59, 59 ) }
timeframe

{'start': datetime.datetime(2018, 9, 1, 0, 0),
 'end': datetime.datetime(2018, 11, 7, 23, 59, 59)}

In [ ]:
# get time series
response = client.getTimeSeries ( bbox, timeframe, resolution )
response._df

In [ ]:
import matplotlib.pyplot as plt

_, axes = plt.subplots( nrows=len(response._df), ncols=2, figsize=(20,120) )

for idx, row in response._df.iterrows():
    axes[ idx ][ 0 ].imshow( row[ 'AGB.tif'] )
    axes[ idx ][ 1 ].imshow( row[ 'NDVI.tif'], vmin=-1.0 )